In [ ]:
!pip install -q -U datasets > /dev/null

In [ ]:
!pip install transformers

In [ ]:
!pip install sentencepiece

In [ ]:
from transformers import AutoModelWithLMHead, AutoTokenizer
from datasets import load_dataset
import random, warnings
warnings.filterwarnings("ignore")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-wikiSQL")
model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-wikiSQL")

In [ ]:
def get_sql(query):
    
    input_text = "translate English to SQL: %s </s>" % query
    
    features = tokenizer([input_text], return_tensors='pt')

    output = model.generate(input_ids=features['input_ids'], 
               attention_mask=features['attention_mask'])

    return tokenizer.decode(output[0])

In [ ]:
valid_dataset = load_dataset('wikisql', split='validation')

In [ ]:
valid_dataset[0]

In [ ]:
for idx in random.sample(range(len(valid_dataset)), 250):
    print(f"Text: {valid_dataset[idx]['question']}")
    print(f"Pred SQL: {get_sql(valid_dataset[idx]['question'])}")
    print(f"True SQL: {valid_dataset[idx]['sql']['human_readable']}\n")

In [ ]:
!pip install transformers

In [ ]:
!pip install sentencepiece

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [ ]:
tokenizer = T5Tokenizer.from_pretrained('t5-small')

In [ ]:
!pip install datasets

# **Translate FR to ENG**

In [2]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 7.2 MB/s 
     |████████████████████████████████| 182 kB 48.8 MB/s 
     |████████████████████████████████| 115 kB 43.9 MB/s 
     |████████████████████████████████| 212 kB 44.4 MB/s 
     |████████████████████████████████| 127 kB 50.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [1]:
from datasets import load_dataset

ModuleNotFoundError: ignored

In [ ]:
books = load_dataset("opus_books", "en-fr")

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
books = books["train"].train_test_split(test_size=0.2)

In [ ]:
books["train"][0]

{'id': '46701',
 'translation': {'en': '"Why, then," cried Milady, with an incredible tone of truthfulness, "you are not his accomplice; you do not know that he destines me to a disgrace which all the punishments of the world cannot equal in horror?"',
  'fr': "-- Mais, s'écria Milady avec un incroyable accent de vérité, vous n'êtes donc pas son complice, vous ne savez donc pas qu'il me destine à une honte que tous les châtiments de la terre ne sauraient égaler en horreur?"}}

In [ ]:
!pip install transformers

In [ ]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("t5-small")

In [ ]:
source_lang = "fr"
target_lang = "en"
prefix = "translate French to English: "

In [ ]:
def preprocess_function(examples):
    inputs = [prefix + example[source_lang] for example in examples["translation"]]
    targets = [example[target_lang] for example in examples["translation"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs

In [ ]:
tokenized_books = books.map(preprocess_function, batched=True)

  0%|          | 0/102 [00:00<?, ?ba/s]

  0%|          | 0/26 [00:00<?, ?ba/s]

In [ ]:
tokenized_books

DatasetDict({
    train: Dataset({
        features: ['id', 'translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 101668
    })
    test: Dataset({
        features: ['id', 'translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 25417
    })
})

In [ ]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [ ]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    #fp16=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_books["train"],
    eval_dataset=tokenized_books["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

ValueError: ignored

In [ ]:
#sentences = ["The house is wonderful.", "I like to work in NYC."]
sentences = ["Jes suis Hassan.", "Je suis Hassan."]

inputs = tokenizer([prefix + sentence for sentence in sentences], return_tensors="pt", padding=True)

In [ ]:
output_sequences = model.generate(
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    do_sample=False,  # disable sampling to test if batching affects output
)

print(tokenizer.batch_decode(output_sequences, skip_special_tokens=True))

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1364: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


['Jes suis Hassan.', 'Je suis Hassan.']


# **FR to EN translation using deep_translator**

In [ ]:
!pip install deep-translator

In [ ]:
from deep_translator import GoogleTranslator
to_translate = 'je suis Hassan'
translated = GoogleTranslator(source='auto', target='en').translate(to_translate)

In [ ]:
translated